In [1]:
!pip install tensorflow-gpu==2.1.0

     |████████████████████████████████| 421.8 MB 4.8 kB/s 
     |████████████████████████████████| 50 kB 6.5 MB/s 
     |████████████████████████████████| 448 kB 65.3 MB/s 
     |████████████████████████████████| 3.8 MB 36.0 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=5712521621da1f6fb83ef8a737329f6d94460de9c8965f7f7eac1e4be4131851
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Succe

In [2]:
try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Layer

In [9]:
class IdentityBlock(tf.keras.Model):
  def __init__(self, filters, kernel_size):
    super(IdentityBlock, self).__init__(name="")
    self.conv1 = tf.keras.layers.Conv2D(filters, kernel_size, padding="same")
    self.bn1 = tf.keras.layers.BatchNormalization() 
    
    self.conv2 = tf.keras.layers.Conv2D(filters, kernel_size, padding="same")
    self.bn2 = tf.keras.layers.BatchNormalization()

    self.act = tf.keras.layers.Activation("relu")

    self.add = tf.keras.layers.Add()  

  def call(self, input_tensor):
    x = self.conv1(input_tensor)
    x = self.bn1(x) 

    x = self.act(x)

    x = self.conv2(x) 

    x = self.bn2(x) 

    x = self.add([x, input_tensor])

    x = self.act(x) 

    return x

In [10]:
class Resnet(tf.keras.Model):

  def __init__(self, num_classes):

    super(Resnet, self).__init__()

    self.conv = tf.keras.layers.Conv2D(64, 7, padding="same")

    self.bn = tf.keras.layers.BatchNormalization()

    self.act = tf.keras.layers.Activation("relu")

    self.max_pool = tf.keras.layers.MaxPool2D((3,3))

    self.id1a = IdentityBlock(64, 3) 

    self.id1b = IdentityBlock(64, 3)  

    self.global_pool = tf.keras.layers.GlobalAveragePooling2D() 

    self.classifier = tf.keras.layers.Dense(num_classes, activation="softmax")

  def call(self, inputs):
    x = self.conv(inputs)

    x= self.bn(x) 

    x = self.act(x) 

    x = self.max_pool(x) 

    x = self.id1a(x) 

    x = self.id1b(x) 

    x = self.global_pool(x) 

    return self.classifier(x)

In [11]:
def preprocess(features):

  return tf.cast(features["image"], tf.float32)/255., features["label"]


resnet = Resnet(10)

resnet.compile(
    optimizer='adam', 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
)

dataset = tfds.load("mnist", split=tfds.Split.TRAIN, data_dir='./data')

dataset = dataset.map(preprocess).batch(32)

resnet.fit(dataset, epochs=1)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.



Dl Completed...:   0%|          | 0/4 [00:00<?, ? file/s]


Dataset mnist downloaded and prepared to ./data/mnist/3.0.1. Subsequent calls will reuse this data.
1875/1875 [==============================] - 264s 141ms/step - loss: 0.1277 - accuracy: 0.9656
